# HTB Dataset Pre-processing for homework tasks 🔨
for now, it only retrieves and tokenizes the sentences from the HTB dataset.

This notebook transforms the HTB dataset from https://github.com/UniversalDependencies/UD_Hebrew-HTB, into what you need for the verb-prediction assignment. You may equally copy the code into `.py` file and run it directly outside the jupyter environment here (in a command prompt).

### Downloading the right version of the HTB
To future-proof, we standardize on a specific version of the HTB. <br>Those with a keen eye will notice we use here the quick-and-dirty ⚡ way of launching OS commands from directly within the notebook. <br>⚙ Anyway, make sure you have git installed and working on your OS before proceeding

In [1]:
!git clone https://github.com/UniversalDependencies/UD_Hebrew-HTB
!cd UD_Hebrew-HTB && git checkout 82591c955e86222e32531336ff23e36c220b5846

Cloning into 'UD_Hebrew-HTB'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 191 (delta 9), reused 15 (delta 5), pack-reused 169
Receiving objects: 100% (191/191), 8.93 MiB | 3.17 MiB/s, done.
Resolving deltas: 100% (106/106), done.
Checking connectivity... done.
Note: checking out '82591c955e86222e32531336ff23e36c220b5846'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 82591c9... Updated treebank evaluation.


the code cell above is not [idempotent](https://www.wikiwand.com/en/Idempotence) because it fails when run more than once 💥 it's oftentimes good practice to have notebooks completely idempotent. What would be two entirely differnt ways of making that cell idempotent? 💡 

In [2]:
train_set = 'UD_Hebrew-HTB/he_htb-ud-train.conllu'

sentences = []
with open(train_set, 'r') as f:
    for line in f:
        if line.startswith('# text = '):
            sentences.append(line[len('# text = '):])
            
print('{} sentences read'.format(len(sentences)))

5241 sentences read


### Observing what we got

In [3]:
import random
random.sample(sentences, 12)

['הוא קרא דרור ליצרי הנקמה הכי כמוסים.\n',
 'נעשות שם עוד מאות סדרות שהן ירודות ביותר, ולכן ההשוואה אינה הוגנת".\n',
 'הרי ככלות הכל המלט היה סקנדינווי.\n',
 'ההנהלה אף דנה בהכנת מסמך בקשה לסיוע חירום מהממשלה.\n',
 'והנה, מאז פרץ מה שפרץ במפרץ, אין שבוע שבו לא קושרים הצופרים במקום כלשהו קשר אפל לקצר את חיי התושבים.\n',
 'לא אחת, כדי לא לסכן את סיכויי הנישואים של הבנות, מעדיפים האבות מין אוראלי ואנאלי.\n',
 'העיתון מוסר, מפי עדים במחלקת העבודות המיוחדות, כי א-נוסייר הגיב בזעם כששמע על ההעברה.\n',
 'נמסר על רימון גז שניפץ שמשתה של מכונית.\n',
 'הגנרל נורמן שוורצקוף, מפקד הגייסות האמריקאיים הנערכים מול עיראק, מעריך כי "הכוחות שבפיקודו מסוגלים למחוק את עיראק מן המפה".\n',
 'בגיל 61 עזב רשמית את בית הספר והחל להתפרנס כנהג מונית, חוטב עצים, משיט רפסודה בנהר ונהג משאית.\n',
 'אני גם מסכים בהחלט שלצד העניין הלאומי צריך לשים דגש חזק יותר על הלקח ההומניסטי: דיכוי הגזענות, ביצור הדמוקרטיה וכו.\n',
 'מאחר שבכל דלת יש 21 חיבורים, ברור שלאיכות העבודה, כמו גם לטיב הדבקים, יש חלק מכריע בטיב המטבח שרוכ

## Cleaning

We don't really need the newline characters this time around, so we'll drop them. <br>but do all sentences in our data end with a newline? are newline characters used also within sentences?

In [4]:
for sentence in sentences:
    if not sentence.endswith('\n'):
        print(sentence)
        
for sentence in sentences:
    if sentence.index('\n') < len(sentence)-1:
        print(sentence)

okay, so we remove the end of line chars.
<br>we do not remove period characters, because they are not the only end of sentence character allowed in canonical language (which ones are allowed?). read on.

In [5]:
sentences = [sentence[:-1] for sentence in sentences] # this pythonic syntax is called list comprehensions, BTW

In [6]:
random.sample(sentences, 12)

['מעל למיטתו תלוי תצלום קטן של האפיפיור.',
 'באופן פרדוקסלי, נוצר מצב שבו ארה"ב מתחמקת מלקיים דיאלוג ברמה האסטראטגית דווקא עם המדינה היחידה במזרח התיכון עמה יש לה הסכם חתום לשיתוף פעולה אסטראטגי.',
 'תמיכת גאנדי מבטיחה לשקאר 280 קולות בפרלמנט.',
 'בשביל הישיבה היה יותר קל לגייס כספים.',
 'הבנות בעיקר סובלות מ"ציפיות סותרות" של הוריהן, אומר היועץ הפסיכולוגי בבתי-ספר בברלין, גומהור טקבאסרן: המסר הוא "אסור לך לבלוט אבל את חייבת להישאר טורקייה" (לבלוט בסביבה ולעורר את לעגה על-ידי בגדים הנחשבים צנועים ומטפחת ראש), סתירה שאינה ניתנת ליישוב.',
 'הוא האשים את משרד האוצר בעיכוב העברת 80 מיליון ש"ח למקורות כסובסידיה לשנת 90 וכתשלום על מפעלי מים שהחברה הקימה השנה.',
 'באופיו היה כהנא איש אלים ומתפרץ.',
 'היא נרצחה בידי אחיה.',
 'מכבי רמת גן שמט ניצחון אחרי שהוביל במרבית דקות המשחק.',
 'המשלחת כוללת שלושה משקולאים.',
 'התמונות ששש מהן מופיעות בספר מגרות את הזיותיו המיניות של דון ריגוברטו.',
 'הנאשם התקשר בארץ עם עולה מאתיופיה, גרושה ואם לשני ילדים.']

So rather than remove periods and question/exclamation marks, we make them a token in their own right, wherever encountered, much like the treatment we give other punctuation characters when tokenizing text. 

## Tokenization
Tokenizing is not the most exciting nor a trivial task; some characters just carry ambiguity, for example:

+ the dot character (`.`) has different uses:
    + the dog barked.
    + 0.33
    + David C.
    + Sept. 2000
    + M.D.
    + Mr.
    + Mrs.
    + SKU 3343.22.CE
    <br><br>
+ a hebrew quote character (`"`) has different uses:
    + "הוא אמר "לא הבנתי
    + ד"ר שקשוקה
    + "היא אמרה ש"לא
    <br><br>

+ this also holds for other and additional characters ― depending on the language, language register, and even on written locale conventions mainly concerning the writing of numbers and dates.

<br>We will not discuss tokenization yet, but rather use a ready made tokenization function pre-supplied. You may browse it to see what's involved, and generally, other approaches for tokenization abound.

In [7]:
from hltk.tokenization.tokenizer import tokenize

tokenized = list(map(tokenize ,sentences))
random.sample(tokenized, 10)

[['בשבועיים',
  'האחרונים',
  'כבר',
  'ראיין',
  'שר',
  'המשטרה',
  'כעשרה',
  'קציני',
  'משטרה',
  'מנוסים',
  'וצעירים',
  'יחסית',
  'בדרגת',
  'ניצב',
  'משנה',
  ',',
  'מקרב',
  'שדרת',
  'הביניים',
  'של',
  'פיקוד',
  'המשטרה',
  ',',
  'כמועמדים',
  'לאייש',
  'את',
  'יחידת',
  'המטה',
  'המקצועית',
  'שלו',
  '.'],
 ['רק',
  'אחרי',
  'שראיתי',
  'את',
  'הסרט',
  'ענק',
  'עם',
  'גיימס',
  'דין',
  'הבנתי',
  'מאיפה',
  'מארק',
  'לקח',
  'את',
  'המאניירות',
  'ואת',
  'המימיקה',
  'שלו',
  '.'],
 ['לתנועת',
  'המחאה',
  'הצטרפו',
  'גם',
  'האיגודים',
  'המקצועיים',
  'של',
  'הטלוויזיה',
  '.'],
 ['במקביל',
  ',',
  'פנו',
  'כמה',
  'מיצרני',
  'הרהיטים',
  'למשרד',
  'התעשייה',
  'והמסחר',
  'בדרישה',
  'שלא',
  'להכין',
  'תקן',
  'כזה',
  '.'],
 ['פרויקטים',
  'חדשים',
  'במחקר',
  'ופיתוח',
  'בהיקף',
  'כולל',
  'של',
  'שמונה',
  'מיליון',
  'דולר',
  ',',
  'המשותפים',
  'לחברות',
  'ישראליות',
  'ואמריקאיות',
  ',',
  'יוגשו',
  'למועצת',
  'המנהלים',
  'של'

***Question***: What would've happened hadn't we removed the newline characters first? and could we, very alternatively, defer treating them to a later stage rather than cleaning them away first?

You may have possibly noticed in the foregoing, that for short sentences, jupyter may display the Hebrew text LTR rather than RTL (from left to right rather than from right to left), or just garble the order of words with regard to punctuation. Go back and check. 

This phenomena is due to at least two key factors:

+ Any component in the application stack, the specific browser implementation or locale settings, the web page's code or front-end frameworks used, or in badly-written cases the application itself, may tweak or retrofit letter order and/or word order, as well as decide the proper horizontal alignment. 
<br>Practically, _printing one word per line_ entirely side-steps this aspect (in a cross-platform way!), at least for languages strictly characterized as keeping a horizontal writing order. <br><br>

+ Additionally, [perhaps even the most standardized algorithm for handling mixed language texts](https://en.wikipedia.org/wiki/Bi-directional_text?Unicode_bidi_support), a.k.a bidirectional text, will not always work as intended, because bi-directional text can be by definition ambiguous (think, why? in what cases? we will deeply study a concept you still don't have at this stage of the course, that may help to come up with an answer). <br><br>And so, the layer of display logic in charge of multi-lingual text display, may actually garble indentation e.g. with RTL text when assuming an LTR default, as the display stack in your browser works to figure how to display your text. 

<br>*it is interesting to note, that in spoken text we _do not have this problem at all_, because the sounds of speech are always uttered in their natural order along the time axis, regardless of the languages involved. It is in writing, that the ordering transforms from being temporal to being gemoetric, thusly giving rise to these issues when different-directionality languages are interleaved in text.
